# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [93]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [94]:
import numpy as np
import pandas as pd
import sys
import os
import re
from sqlalchemy import create_engine
import pickle
from scipy.stats import gmean
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

In [95]:
# load data from database
database_name = "../data/DisasterResponse.db"
table_name = "disaster_response_table"
engine = create_engine('sqlite:///'+database_name)
df = pd.read_sql_table(table_name, engine)
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      26216 non-null  int64 
 1   message                 26216 non-null  object
 2   original                10170 non-null  object
 3   genre                   26216 non-null  object
 4   related                 26216 non-null  int64 
 5   request                 26216 non-null  int64 
 6   offer                   26216 non-null  int64 
 7   aid_related             26216 non-null  int64 
 8   medical_help            26216 non-null  int64 
 9   medical_products        26216 non-null  int64 
 10  search_and_rescue       26216 non-null  int64 
 11  security                26216 non-null  int64 
 12  military                26216 non-null  int64 
 13  child_alone             26216 non-null  int64 
 14  water                   26216 non-null  int64 
 15  fo

In [97]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [98]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
col_to_removed = []
for key, value in df.describe().items():
    if value['max']<1:
        col_to_removed.append(key)
print(col_to_removed)

['child_alone']


In [12]:
cleaned_df = df.drop(col_to_removed, axis=1)
print(df.shape)
cleaned_df.shape

(26216, 40)


(26216, 39)

In [13]:
X = cleaned_df['message']
Y = cleaned_df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [99]:
def tokenize(text, url_place_holder_string="urlplaceholder"):
    """
    This function is used to convert the text into a list of words
    
    :param text: input texts
    :param url_place_holder_string: this word will replace the urls texts
    """
     # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

In [100]:
temp1 = tokenize("I want to fly in the air!")
print(temp1)

['i', 'want', 'to', 'fly', 'in', 'the', 'air', '!']


In [103]:
temp1 = tokenize("I want to fly in https://www.163.com !")
print(temp1)

['i', 'want', 'to', 'fly', 'in', 'urlplaceholder', '!']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [105]:
pipeline_basic = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('count', CountVectorizer(tokenizer=tokenize)),
                ('tfid', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [106]:
X.iloc[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [108]:
y=Y.astype('bool')

In [109]:
y.iloc[0]

related                    True
request                   False
offer                     False
aid_related               False
medical_help              False
medical_products          False
search_and_rescue         False
security                  False
military                  False
child_alone               False
water                     False
food                      False
shelter                   False
clothing                  False
money                     False
missing_people            False
refugees                  False
death                     False
other_aid                 False
infrastructure_related    False
transport                 False
buildings                 False
electricity               False
tools                     False
hospitals                 False
shops                     False
aid_centers               False
other_infrastructure      False
weather_related           False
floods                    False
storm                     False
fire    

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [111]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((19662,), (19662, 36), (6554,), (6554, 36))

In [112]:
pipeline_fitted = pipeline_basic.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [113]:
# calculate the predictions
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

In [116]:
 
print(y_test.values.shape)
print(y.columns.values.shape)
 


(6554, 36)
(36,)


In [117]:
print(classification_report(y_test.values.astype('bool'), 
                      y_prediction_test.astype('bool'), target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5025
               request       0.78      0.54      0.64      1130
                 offer       0.00      0.00      0.00        33
           aid_related       0.78      0.60      0.68      2756
          medical_help       0.63      0.28      0.39       536
      medical_products       0.65      0.31      0.42       354
     search_and_rescue       0.55      0.15      0.24       203
              security       0.32      0.10      0.15       125
              military       0.67      0.41      0.51       224
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.62      0.68       426
                  food       0.82      0.68      0.74       753
               shelter       0.80      0.52      0.63       597
              clothing       0.79      0.37      0.50       100
                 money       0.59      

/root/miniconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.6/site-packages/sklearn/metr

### 6. Improve your model
Use grid search to find better parameters. 

In [119]:
 
parameters_grid = {'classifier__estimator__learning_rate': 
                    [0.01, 0.02, 0.05, 0.1, 0.2],
                   'classifier__estimator__n_estimators': 
                    [10, 30, 50, 80]
                  }
cv = GridSearchCV(pipeline_basic, 
                  param_grid=parameters_grid, 
                  scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f1bf5cc1d90>)),
                                                                                        ('tfid',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.01, 0.02,
                                                                  0.05, 0.1,
                                                             

In [122]:
cv.best_params_

{'classifier__estimator__learning_rate': 0.2,
 'classifier__estimator__n_estimators': 80}

In [124]:
cv.best_estimator_

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('count',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f1bf5cc1d90>)),
                                                                 ('tfid',
                                                                  TfidfTransformer())]))])),
                ('classifier',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(learning_rate=0.2,
                                                                    n_estimators=80)))])

In [125]:
cv.best_score_

0.629576577093133

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [120]:
y_prediction_test = cv.predict(X_test)
print(classification_report(y_test.values.astype('bool'), y_prediction_test.astype('bool'), target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.79      0.99      0.87      5025
               request       0.84      0.44      0.58      1130
                 offer       0.00      0.00      0.00        33
           aid_related       0.80      0.50      0.61      2756
          medical_help       0.66      0.10      0.17       536
      medical_products       0.70      0.18      0.28       354
     search_and_rescue       0.82      0.07      0.13       203
              security       0.50      0.01      0.02       125
              military       0.74      0.14      0.24       224
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.59      0.67       426
                  food       0.82      0.70      0.76       753
               shelter       0.86      0.45      0.59       597
              clothing       0.76      0.28      0.41       100
                 money       0.71      

/root/miniconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.6/site-packages/sklearn/metr

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [126]:
pipeline_random_forest = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count', CountVectorizer(tokenizer=tokenize)),
                ('tfid', TfidfTransformer())
            ])),
        ])),

        ('classifier', MultiOutputClassifier(RandomForestClassifier()))
    ])

pipeline_fitted = pipeline_random_forest.fit(X_train, y_train)

#y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      5025
               request       0.90      0.41      0.56      1130
                 offer       0.00      0.00      0.00        33
           aid_related       0.80      0.60      0.69      2756
          medical_help       0.61      0.04      0.08       536
      medical_products       0.73      0.03      0.06       354
     search_and_rescue       0.69      0.04      0.08       203
              security       0.33      0.01      0.02       125
              military       0.91      0.04      0.09       224
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.19      0.32       426
                  food       0.91      0.34      0.50       753
               shelter       0.89      0.25      0.39       597
              clothing       0.71      0.05      0.09       100
                 money       1.00      

/root/miniconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.6/site-packages/sklearn/metr

### 9. Export your model as a pickle file

In [128]:
model_file_name ='classifier.pkl'
with open(model_file_name, "wb") as f:
    pickle.dump(pipeline_fitted, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.